## Importación de librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import gc
from sklearn.ensemble import RandomForestClassifier

import time
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.externals import joblib
from sklearn.metrics import confusion_matrix, classification_report

## Lectura del dataset

In [2]:
Consolidado=pd.read_csv('Dataset_consolidado.csv', error_bad_lines=False,warn_bad_lines=False,low_memory=False)

#### Muestra del dataset

In [3]:
Consolidado.head()

,seg_str_EMPRENDEDOR,seg_str_OTRO,seg_str_PERSONAL,seg_str_PERSONAL PLUS,seg_str_PREFERENCIAL,ocupacion_1,ocupacion_2,ocupacion_3,ocupacion_4,ocupacion_5,...,nivel_academico_I,nivel_academico_N,nivel_academico_P,nivel_academico_S,nivel_academico_T,nivel_academico_U,fecha,hora,valor_trx,etiqueta
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,20171009.0,90609.0,15100.91,1.0
1,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,20171013.0,101444.0,157299.02,2.0
2,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,20170825.0,175028.0,371718.55,2.0
3,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,20180124.0,104113.0,382742.66,2.0
4,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,20180325.0,153255.0,125253.38,2.0


#### Número de registros del dataset

In [4]:
Consolidado.shape

(3197091, 127)

#### Número de registros del dataset por etiqueta

In [5]:
Consolidado.etiqueta.value_counts()

2.0    1710777
1.0    1450457
3.0      35857
Name: etiqueta, dtype: int64

#### Extracción de una porción de los datos
Para balancear la muestra

se utiliza la funcion shuffle con el fin de tomar distintos registros cada vez que se ejecuta el notebook

In [6]:
Consolidado=pd.concat([shuffle(Consolidado[Consolidado.etiqueta==1]).iloc[0:35857],shuffle(Consolidado[Consolidado.etiqueta==2]).iloc[0:35857],shuffle(Consolidado[Consolidado.etiqueta==3])])

In [7]:
Consolidado.etiqueta.value_counts()

3.0    35857
2.0    35857
1.0    35857
Name: etiqueta, dtype: int64

#### Asignación de variable predictora y variable a predecir

In [8]:
y=Consolidado.etiqueta

In [9]:
X=Consolidado.iloc[:, Consolidado.columns != 'etiqueta']

#### liberación de memoria

In [10]:
del Consolidado

In [11]:
gc.collect()

7

#### Partición de los datos en entrenamiento y prueba

In [13]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.25,stratify=y)

In [14]:
X_train2.shape

(80678, 126)

#### Esquema de parámetros del modelo

In [15]:
RandomForestClassifier()

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## Definición del Modelo
Al dataset se le incorpora una reducción de dimensionalidad, luego se implementa el modelo RandomForestClassifier para clasificación multiclase. Finalmente se aplica GridSearchCV, el cual basado en validación cruzada y el score balanced_accuracy encuentra el mejor modelo dado por los diferentes hiperparametros.

Se elige balanced_accuracy como medición del score para modelos multiclase desbalanceados. 

In [16]:
parametros = {"pca__n_components": [3,5,7,8,10],
              "rf__n_estimators": [5,10,15],
              "rf__criterion": ["gini","entropy"],
              "rf__max_depth": [None,10]}

pipeline = Pipeline(steps=[('pca', PCA()),
                           ('rf', RandomForestClassifier())])


score='balanced_accuracy'

grid_search_LP=GridSearchCV(pipeline, param_grid=parametros,cv=10,scoring=score,n_jobs=-1)

#### Ajuste del modelo

In [17]:
tiempo_inicio=time.clock()
grid_search_LP.fit(X_train2,y_train2)
tiempo_final=time.clock()
print("tiempo de ejecucion : "+str(int(tiempo_final-tiempo_inicio))+" segundos")

C:\Users\lssro\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


tiempo de ejecucion : 781 segundos


C:\Users\lssro\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


#### Mejores parámetros encontrados 
con los que se construye el modelo

In [18]:
grid_search_LP.best_params_

{'pca__n_components': 10,
 'rf__criterion': 'entropy',
 'rf__max_depth': None,
 'rf__n_estimators': 15}

#### Selección del mejor modelo
Según los hiperparámetros encontrados

In [19]:
grid_search_LP.best_estimator_

Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
         ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

#### Score del mejor modelo

In [20]:
grid_search_LP.best_score_

0.8519921019873136

#### Matriz de confunsión
En la cual se analiza el desempeño del modelo según el número de predicciones de cada clase dadas por: VP, VN, FP, FN

In [21]:
confusion_matrix(y_test2, grid_search_LP.best_estimator_.predict(X_test2))

array([[7664,  788,  512],
       [ 674, 7808,  483],
       [ 755,  677, 7532]], dtype=int64)

#### Reportes del score del modelo
Basado en un comparativo de métricas 

In [23]:
result=classification_report(y_test2, grid_search_LP.best_estimator_.predict(X_test2))

In [24]:
print(result)

              precision    recall  f1-score   support

         1.0       0.84      0.85      0.85      8964
         2.0       0.84      0.87      0.86      8965
         3.0       0.88      0.84      0.86      8964

   micro avg       0.86      0.86      0.86     26893
   macro avg       0.86      0.86      0.86     26893
weighted avg       0.86      0.86      0.86     26893



# Modelo Adaboost Classifier

## Importación de Librerías

In [51]:
from sklearn.tree import DecisionTreeClassifier

In [52]:
from sklearn.ensemble import AdaBoostClassifier

## Definición del Modelo
Al dataset se le incorpora una reducción de dimensionalidad, luego se implementa el modelo AdaBoostClassifier el cual es un tipo de modelo de ensamble en el cual se basa en etimadores DecisionTreeClassifier. El cual por medio de pesos ajusta las instancias clasificadas incorrectamente de modo que las clasificaciones posteriores se centren más en los errores cometidos.
Finalmente se aplica GridSearchCV, el cual basado en validación cruzada y el score balanced_accuracy encuentra el mejor modelo dado por los diferentes hiperparametros.

Se elige balanced_accuracy como medición del score para modelos multiclase desbalanceados. 

In [61]:
parametros = {"pca__n_components": [3,5,7,8,10],
              "ada__n_estimators": [500,600,700],
              "ada__learning_rate": [1.1,1.3,1.5,1.7]}

pipeline = Pipeline(steps=[('pca', PCA()),
                           ('ada',  AdaBoostClassifier(DecisionTreeClassifier(max_depth=2)))])


score='balanced_accuracy'

grid_search_LP=GridSearchCV(pipeline, param_grid=parametros,cv=10,scoring=score,n_jobs=-1)

#### Ajuste del modelo

In [62]:
tiempo_inicio=time.clock()
grid_search_LP.fit(X_train2,y_train2)
tiempo_final=time.clock()
print("tiempo de ejecucion : "+str(int(tiempo_final-tiempo_inicio))+" segundos")

C:\Users\lssro\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


tiempo de ejecucion : 2338 segundos


C:\Users\lssro\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


### Mejores parámetros encontrados 
con los que se construye el modelo

In [63]:
grid_search_LP.best_params_

{'ada__learning_rate': 1.1, 'ada__n_estimators': 700, 'pca__n_components': 10}

#### Selección del mejor modelo
Según los hiperparámetros encontrados

grid_search_LP.best_estimator_

#### Score del mejor modelo

In [65]:
grid_search_LP.best_score_

0.6040831401627674

#### Matriz de confunsión
En la cual se analiza el desempeño del modelo según el número de predicciones de cada clase dadas por: VP, VN, FP, FN

In [66]:
confusion_matrix(y_test2, grid_search_LP.best_estimator_.predict(X_test2))

array([[819, 271, 255],
       [258, 827, 259],
       [272, 316, 757]], dtype=int64)

#### Reportes del score del modelo
Basado en un comparativo de métricas

In [67]:
result=classification_report(y_test2, grid_search_LP.best_estimator_.predict(X_test2))

In [68]:
print(result)

              precision    recall  f1-score   support

         1.0       0.61      0.61      0.61      1345
         2.0       0.58      0.62      0.60      1344
         3.0       0.60      0.56      0.58      1345

   micro avg       0.60      0.60      0.60      4034
   macro avg       0.60      0.60      0.60      4034
weighted avg       0.60      0.60      0.60      4034

